# Pipelines

Pipelines sequentially apply **a list of transforms** and a **final estimator**. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.


### Summary

* Pipeline of transforms with a final estimator.

* Sequentially apply a list of transforms and a final estimator. 

    * Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. 
    * The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.

* The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

## Simple Pipeline

The simple pipeline is composed of the folloing steps:
- Transformation
    - Scaling values between 0 and 1
    - PCA (we keep 2 components)
- Estimator
    - Logistic Regression

### Transformation


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

preprocessing_transformer = Pipeline(steps=[('scale_01', MinMaxScaler(feature_range=(0, 1))), 
                                            ('PCA', PCA(n_components=2))])

In [5]:
preprocessing_transformer

Pipeline(memory=None,
         steps=[('scale_01', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('PCA',
                 PCA(copy=True, iterated_power='auto', n_components=2,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False))],
         verbose=False)

### Estimator

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', multi_class='auto')

### Creating and evaluating the Pipeline

In [7]:
from sklearn.metrics import accuracy_score

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessing_transformer', preprocessing_transformer),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

[Pipeline]  (step 1 of 2) Processing preprocessing_transformer, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s
Accuracy Score: 0.8666666666666667


### Analyzing the transformation

In [8]:
transformed_Dataset = preprocessing_transformer.fit_transform(X_train)

In [9]:
type(transformed_Dataset)

numpy.ndarray

In [10]:
tra_df = pd.DataFrame(transformed_Dataset)

In [11]:
tra_df.shape

(120, 2)

In [12]:
tra_df.head()

,0,1
0,0.392210,0.048456
1,0.090487,-0.089585
2,-0.629042,0.133227
3,0.297395,-0.016936
4,0.525822,-0.071939


In [13]:
# Simple check

scaled = MinMaxScaler(feature_range=(0, 1))
scaled_X_train = scaled.fit_transform(X_train)
pcaed = PCA(n_components=2)
pca_X_train = pcaed.fit_transform(scaled_X_train)

In [14]:
pca_X_train

array([[ 3.92209763e-01,  4.84561075e-02],
       [ 9.04869356e-02, -8.95853266e-02],
       [-6.29042414e-01,  1.33227096e-01],
       [ 2.97394765e-01, -1.69364128e-02],
       [ 5.25821952e-01, -7.19394322e-02],
       [-8.81287220e-03, -2.16744469e-01],
       [-5.36677344e-01,  3.00755516e-01],
       [ 2.68771542e-01, -1.40752213e-01],
       [ 1.18143450e-01, -2.69059275e-02],
       [ 2.51091885e-02, -1.81673754e-01],
       [ 5.25692275e-01,  5.32019212e-02],
       [-6.94577142e-01, -3.59042800e-02],
       [ 5.43323143e-01,  1.04376017e-01],
       [-5.34922624e-01,  1.01994571e-01],
       [-6.15543324e-01,  2.31944792e-01],
       [-1.46419376e-01, -4.91909893e-01],
       [ 4.09356632e-01,  2.26626713e-02],
       [ 6.26859126e-01,  1.45225238e-01],
       [ 2.57239670e-01, -3.25774678e-01],
       [ 4.91330874e-01, -1.45418497e-01],
       [-3.11137441e-02, -2.39957062e-01],
       [ 7.51055857e-01,  1.48508623e-01],
       [ 2.30644832e-01,  1.25073917e-01],
       [-4.

## ColumnTransformer: Managing different kinds of transformers on different columns:
Extracted and extended from a kaggle.com tutorial

Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space. This is useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer.

In [16]:
dataset = pd.read_csv("data/melb_data.csv")
dataset.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(dataset.iloc[:,1:-1], dataset.iloc[:,-1], train_size=0.8, test_size=0.2,
                                                      random_state=0)

We construct the full pipeline in three steps.
* Step 1: Define Preprocessing Steps
* Step 2: Define the Model
* Step 3: Create and Evaluate the Pipeline

### Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to bundle together different preprocessing steps. The code below:
- imputes missing values in numerical data, and
- imputes missing values and applies a one-hot encoding to categorical data.

In [22]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]
print(categorical_cols)
print(numerical_cols)

['Type', 'Method', 'Regionname']
['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']


In [23]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

### Step 2: Define the Model

In [24]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### Step 3: Create and Evaluate the Pipeline

In [25]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 153.0232142857143


### Parameter tuning

Setting parameters of the various steps is enabled by using their names and the parameter name separated by a ‘__’

In [30]:
# Example using a Grid Search
from sklearn.model_selection import GridSearchCV

parameters = {
    'model__n_estimators': [10,50,100],
    'preprocessor__num__strategy': ['most_frequent','constant'],
    'preprocessor__cat__imputer__strategy': ['most_frequent','constant'],
}

gs_clf = GridSearchCV(my_pipeline, parameters, scoring = 'neg_mean_absolute_error', cv=5, iid=False, n_jobs=-1)

gs_clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         SimpleImputer(add_indicator=False,
                                                                                       copy=True,
                                                                                       fill_value=None,
                                                                                       missing_values=nan,
                                       

In [31]:
gs_clf.best_params_

{'model__n_estimators': 50,
 'preprocessor__cat__imputer__strategy': 'most_frequent',
 'preprocessor__num__strategy': 'constant'}

In [28]:
gs_clf.best_score_

-189.78764740041646

## FeatureUnion: Applying multiple transformers in parallel

Concatenates results of multiple transformer objects.

This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import SelectKBest

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=2)

#Normalizing is always a good choice
scaler = MinMaxScaler(feature_range=(0, 1))

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection), ("normal", scaler)])

In [ ]:
# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)
print("Combined space has", X_features.shape[1], "features")

In [ ]:
model = LogisticRegression()


# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('combined_features', combined_features),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

In [ ]:
# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=2)

#Normalizing is always a good choice
scaler = MinMaxScaler(feature_range=(0, 1))

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection), ("normal", scaler)], transformer_weights={"pca":0.4, "univ_select":0.2, "normal":0.3})

In [ ]:
model = LogisticRegression()


# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('combined_features', combined_features),
                              ('model', model)
                             ], verbose = True)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = accuracy_score(y_valid, preds)
print('Accuracy Score:', score)

## FunctionTransformer: Constructs a transformer from an arbitrary callable.

Concatenates results of multiple transformer objects.

This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

In [ ]:
from sklearn.preprocessing import FunctionTransformer

dataset = pd.read_csv("melb_data.csv")
X_train, X_valid, y_train, y_valid = train_test_split(dataset.iloc[:,1:-1], dataset.iloc[:,-1], train_size=0.8, test_size=0.2,
                                                      random_state=0)

#Selecting the numerical columns
def columns_num(X):
    numerical_cols = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]
    return X.loc[:,numerical_cols]

fill_na_transformer = Pipeline(steps=[ ('drop_cols', FunctionTransformer(columns_num, validate=False)),
                                       ('fill_na', SimpleImputer(strategy='most_frequent'))  ])


model = RandomForestRegressor(n_estimators=100, random_state=0)



# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', fill_na_transformer),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)